In [14]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_functions
%run My_functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
client = Client()

In [4]:
# Data Import
hourly_raw_data = dd.read_csv("hour.csv", blocksize=16e4)

In [5]:
### Changing the date type
hourly_raw_data["dteday"].map_partitions(
    pd.to_datetime, format="%Y-%m-%d", meta=("datetime64[ns]")
)

Dask Series Structure:
npartitions=8
    datetime64[ns]
               ...
         ...      
               ...
               ...
dtype: datetime64[ns]
Dask Name: to_datetime, 40 tasks

### Adding the temp atemp windspeed hum relative to the last 7 days value


In [8]:
to_relative = ["temp", "atemp", "hum", "windspeed"]
hourly_raw_data = relative_values(hourly_raw_data, to_relative)

### Adding `RushHour-High` &    `RushHour-Med`  &   `RushHour-Low`


In [9]:
hourly_raw_data["RushHour-High"] = 0
hourly_raw_data["RushHour-Med"] = 0
hourly_raw_data["RushHour-Low"] = 0
hourly_raw_data = hourly_raw_data.map_partitions(
    lambda df: df.apply(lambda x: addRushHourFlags(x), axis=1)
)

## Splitting Data

In [10]:
workingdays = num_name(hourly_raw_data.loc[(hourly_raw_data["workingday"].isin([1]))])
holidays = num_name(hourly_raw_data.loc[(~hourly_raw_data["workingday"].isin([1]))])

## Mean of the past 3 weeks during the same hour

In [11]:
# holidays['mean_per_hour'] = 0
# workingdays['mean_per_hour'] = 0
# holidays = holidays.map_partitions(lambda df: df.apply(lambda x: mean_per_hour_3weeks_row(x , holidays), axis=1))
# workingdays = workingdays.map_partitions(lambda df: df.apply(lambda x: mean_per_hour_3weeks_row(x , holidays), axis=1))

### One hot Encoding | 2x for splitted datasets
For `season`, `weathersit`, `mnth`,`weekday`,`hr`

In [12]:
category = ["season", "weathersit", "mnth", "weekday", "hr"]

workingdays = onehot_encode(workingdays, category)

holidays = onehot_encode(holidays, category)

## Genetic Programming | 2x for splitted datasets

In [15]:
dates = workingdays["dteday"]
registered = workingdays["registered"]
casual = workingdays["casual"]
instant = workingdays["instant"]
workingdays = Genetic_P(
    workingdays.drop(["registered", "casual", "dteday", "instant"], axis=1), "cnt"
)
workingdays["instant"] = instant
workingdays["dteday"] = dates
workingdays["registered"] = registered
workingdays["casual"] = casual

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.11         0.100762        2         0.660207         0.643796      1.08m
   1     6.83         0.352862        6         0.731759         0.717723     56.63s
   2     4.48         0.584361        8         0.734132         0.728253     41.67s
   3     5.55         0.590216        7         0.756784         0.726055     32.30s
   4     6.27         0.634432       11         0.774373         0.753799     29.34s
   5     9.18          0.63095       16         0.810326         0.801205     29.13s
   6    12.67         0.649777       18         0.830625         0.821608     28.41s
   7    18.05         0.684644       25         0.853502         0.858503     25.69s
   8    23.90         0.723285       41         0.864501         0.860504  

In [16]:
instant = holidays["instant"]
dates = holidays["dteday"]
registered = holidays["registered"]
casual = holidays["casual"]
holidays = Genetic_P(
    holidays.drop(["registered", "casual", "dteday", "instant"], axis=1), "cnt"
)
holidays["instant"] = instant
holidays["dteday"] = dates
holidays["registered"] = registered
holidays["casual"] = casual

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     7.94           0.1202        3         0.772699         0.760499     22.75s
   1     7.02         0.400414        6         0.776181         0.817996     28.05s
   2     5.17         0.593223        8         0.799674         0.801326     26.36s
   3     4.15         0.693106        7         0.802046         0.773434     25.07s
   4     6.12         0.686993       12         0.817007         0.820478     21.75s
   5     8.47             0.67       22         0.829299         0.838649     22.02s
   6    10.69         0.633926       22         0.832744         0.807734     22.41s
   7    11.96          0.63186       22         0.830962         0.824253     19.01s
   8    13.71         0.609937       22         0.835431         0.781799  

In [26]:
workingdays.compute().to_csv("workingdays_data_prepared.csv", index=False)
holidays.compute().to_csv("holidays_data_prepared.csv", index=False)

In [ ]:
'